# 4. Análise Exploratória de Dados (EDA)

**Objetivo:** Analisar os dados gerados e os resultados da otimização para identificar padrões e oportunidades de melhoria.

Este notebook deve ser executado **após** a geração dos dados (via notebooks anteriores ou via app.py se os CSVs forem salvos).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuração Visual
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Carregamento dos Dados

In [ ]:
try:
    df_orders = pd.read_csv('data/pedidos_backlog.csv')
    df_skus = pd.read_csv('data/mestre_skus.csv')
    
    if os.path.exists('data/mapa_alocacao_otimizado.csv'):
        df_alloc = pd.read_csv('data/mapa_alocacao_otimizado.csv')
    else:
        df_alloc = pd.read_csv('data/mapa_alocacao_inicial.csv')
        
    df_layout = pd.read_csv('data/layout_fisico.csv')
    print("Dados carregados com sucesso de 'data/'.")
except FileNotFoundError:
    print("ERRO: Arquivos não encontrados. Rode a simulação primeiro.")

## 2. Análise de Demanda (Pareto / Curva ABC)

In [ ]:
# Calcular total de itens vendidos por SKU
sku_demand = df_orders.groupby('sku_id')['quantity'].sum().reset_index()
sku_demand = sku_demand.sort_values(by='quantity', ascending=False)

# Calcular % Acumulado
sku_demand['cumulative_percentage'] = sku_demand['quantity'].cumsum() / sku_demand['quantity'].sum()

plt.figure(figsize=(10, 6))
sns.lineplot(data=sku_demand, x=range(len(sku_demand)), y='cumulative_percentage', color='blue')
plt.axhline(0.8, color='red', linestyle='--', label='80% da Demanda')
plt.title('Curva ABC de Demanda (Pareto)')
plt.xlabel('SKUs (Rankeados)')
plt.ylabel('% Acumulado da Demanda')
plt.legend()
plt.show()

## 3. Análise de Alocação Física (Heatmap)

In [ ]:
# Juntar Layout com Alocação
df_viz = df_layout.merge(df_alloc[['bin_id', 'sku_effort']], on='bin_id', how='left')

# Preencher NaNs com 0 (Bins vazios)
df_viz['sku_effort'] = df_viz['sku_effort'].fillna(0)

# Heatmap do Nível 1 (Chão) - Zona A
heatmap_data_A = df_viz[(df_viz['z'] == 1) & (df_viz['zone'] == 'A')].pivot(index="y", columns="x", values="sku_effort")

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data_A, cmap="YlOrRd", cbar_kws={'label': 'Esforço de Picking (Score)'})
plt.title('Mapa de Calor de Esforço - Zona A (Nível 1)')
plt.show()

## 4. Distribuição de Peso dos Pedidos

In [ ]:
# Calcular peso total por pedido
order_weights = df_orders.merge(df_skus[['sku_id', 'weight_per_unit_kg']], on='sku_id')
order_weights['total_line_weight'] = order_weights['quantity'] * order_weights['weight_per_unit_kg']
total_order_weight = order_weights.groupby('order_id')['total_line_weight'].sum()

plt.figure(figsize=(10, 6))
sns.histplot(total_order_weight, bins=30, kde=True)
plt.title('Distribuição do Peso Total dos Pedidos (kg)')
plt.xlabel('Peso (kg)')
plt.show()